#***INTEGRANTES***

* **Isaac Cardena**
* **Jasmin Correa**
* **Javier Martinez**
* **Oscar Polma**

In [ ]:
!pip install pingouin ## necesario instalar pingouin para que no arroje error

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=6fa06b44ed3760427a515a547cb7055b1520a46d3e342786925e2d0f4131bdbc
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


## ***Librerias***



In [ ]:
import statsmodels.api as sm #modelos
import pandas as pd # Para el manejo de datos
import numpy as np # Para operaciones numericas
from scipy import stats
import pingouin as pg ## Lo utilizamos para la prueba de hipotesis de diferencia de medias
import itertools ## Lo utilizamos para la combinacion de diferentes modelo
import statsmodels.api as sm ## Para la estimacion de modelos
from statsmodels.stats.outliers_influence import variance_inflation_factor ## Lo utilizamos para hayar el VIF
from patsy import dmatrices ## Lo utilizamos para el calculo del VIF

## ***Cargar base de datos***

In [ ]:
url = "https://raw.githubusercontent.com/leomanriquezm/introduction-to-econometrics/main/ec1116c_t.csv"

In [ ]:
data = pd.read_csv(url);data

,estrategia,ingresos,mercado_obj,rentabilidad_obj,conectividad,politico,empleados,rentabilidad_int,participacion,eficiencia,deuda
0,3,38,354,7,2,2,649,9,8,8,28
1,2,80,781,18,3,0,3866,14,21,12,91
2,4,91,563,18,3,1,5254,14,34,17,119
3,0,8,10,0,1,3,166,3,10,3,8
4,0,8,58,0,2,1,270,4,18,3,2
...,...,...,...,...,...,...,...,...,...,...,...
1890,4,95,589,15,3,1,4386,13,26,14,80
1891,1,20,123,1,1,3,132,3,6,3,11
1892,3,24,360,3,2,1,652,9,10,5,40
1893,4,105,531,13,4,0,5643,13,18,15,100


# ***Analisis descriptivo variables***

In [ ]:
data.describe()

,estrategia,ingresos,mercado_obj,rentabilidad_obj,conectividad,politico,empleados,rentabilidad_int,participacion,eficiencia,deuda
count,1895.000000,1895.000000,1895.000000,1895.000000,1895.000000,1895.000000,1895.000000,1895.000000,1895.000000,1895.000000,1895.000000
mean,2.056992,48.897098,357.955673,8.341953,2.347757,1.277045,2202.102902,8.621108,15.702902,8.656464,48.936148
std,1.426452,40.254477,279.182212,6.986652,1.214100,1.276369,2222.602695,5.639746,10.200103,5.688563,40.361979
min,0.000000,1.000000,6.000000,0.000000,0.000000,0.000000,50.000000,2.000000,2.000000,2.000000,1.000000
25%,1.000000,11.000000,87.000000,1.000000,1.000000,0.000000,258.000000,3.000000,6.000000,3.000000,11.000000
50%,2.000000,32.000000,293.000000,7.000000,2.000000,1.000000,712.000000,6.000000,14.000000,6.000000,32.000000
75%,3.000000,89.000000,625.000000,15.000000,3.000000,2.000000,4543.000000,14.000000,25.000000,14.000000,89.000000
max,4.000000,120.000000,840.000000,20.000000,4.000000,4.000000,5998.000000,18.000000,34.000000,18.000000,120.000000


## ***Tamaño base de datos dividido por categoria (en base a estrategia)***

In [ ]:
data.groupby('estrategia').size()

estrategia
0    362
1    375
2    366
3    377
4    415
dtype: int64

## ***Media y desviacion estandar variables explicativas***

In [ ]:
data.groupby('estrategia').agg(['mean', 'std'])

ingresos            mercado_obj             rentabilidad_obj  \
                 mean        std        mean         std             mean   
estrategia                                                                  
0            5.256906   2.940362   51.662983   27.814427         0.000000   
1           15.045333   6.328830   94.045333   32.972125         2.488000   
2           94.483607  14.226870  661.117486  105.192973        16.035519   
3           30.156499   6.102791  276.169761   68.964166         6.503979   
4           94.373494  14.962845  670.534940  102.191131        15.792771   

                     conectividad            politico              empleados  \
                 std         mean       std      mean       std         mean   
estrategia                                                                     
0           0.000000     1.013812  0.803267  1.947514  1.479315   177.875691   
1           1.644710     1.504000  0.500652  2.042667  1.377160   277.840000   
2           2.616354     3.535519  0.499420  0.519126  0.500318  4808.767760   
3           2.335118     2.031830  0.801624  1.527851  1.136855   698.108753   
4           2.634855     3.513253  0.500428  0.440964  0.497102  4773.995181   

                       rentabilidad_int           participacion            \
                   std             mean       std          mean       std   
estrategia                                                                  
0            73.652538         3.044199  0.786473     13.392265  6.405135   
1           133.491557         3.448000  0.497953      3.997333  1.371011   
2           696.051036        14.980874  1.999223     25.879781  5.165934   
3            60.590150         5.976127  1.958876      7.976127  1.407413   
4           717.119533        14.954217  1.992819     26.339759  5.005575   

           eficiencia                deuda             
                 mean       std       mean        std  
estrategia                                             
0            2.941989  0.838449   5.160221   2.818964  
1            3.509333  0.500581  15.152000   5.760908  
2           15.081967  1.984557  93.494536  14.520414  
3            5.997347  1.981965  29.992042   6.224304  
4           15.040964  1.952518  95.561446  14.973099

# ***Diferencia de Medias por Variable Explicativa y Media Poblacional*** (n. confianza = 95%)

Realizamos el contraste de hipotesis con pingouin, usando la prueba ***ANOVA***. Ya que la prueba se hara con 5 grupos diferentes. Y el metodo mencionado, es una técnica estadística que se utiliza para comparar la media de tres o más grupos y determinar si existen diferencias significativas entre ellas.
Analizamos las variables de caracter no politomica. Por lo cual, la variable "conectiviadad" y "politico", no se le realizara este contraste.

### ***ingresos***

In [ ]:
#Ho: μ1 = μ2 = μ3 = μ4 = μ5
#Ha: al menos una μi es diferente

pg.anova(data=data, dv='ingresos', between='estrategia', detailed=True).round(3)

# p-value < 0,05

,Source,SS,DF,MS,F,p-unc,np2
0,estrategia,2870409.320,4,717602.330,6826.684,0.0,0.935
1,Within,198671.614,1890,105.117,NaN,NaN,NaN


Existe evidencia estadistica para inferir que al menos una media de la variable ***ingreso*** es estadisticamente diferentes entre si.

In [ ]:
###Categoria 0
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==0]) - 1, loc=np.mean(data[data["estrategia"]==0]["ingresos"]), scale=stats.sem(data[data["estrategia"]==0]["ingresos"]))

(4.952990436454903, 5.56082171824123)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***ingreso*** dado la ***categoria_0***, es estadisticamente diferente de 0

In [ ]:
###Categoria 1
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==1]) - 1, loc=np.mean(data[data["estrategia"]==1]["ingresos"]), scale=stats.sem(data[data["estrategia"]==1]["ingresos"]))

(14.402699510865167, 15.6879671558015)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***ingreso*** dado la ***categoria_1***, es estadisticamente diferente de 0

In [ ]:
###Categoria 2
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==2]) - 1, loc=np.mean(data[data["estrategia"]==2]["ingresos"]), scale=stats.sem(data[data["estrategia"]==2]["ingresos"]))

(93.02122944891296, 95.94598366584113)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***ingreso*** dado la ***categoria_2***, es estadisticamente diferente de 0

In [ ]:
###Categoria 3
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==3]) - 1, loc=np.mean(data[data["estrategia"]==3]["ingresos"]), scale=stats.sem(data[data["estrategia"]==3]["ingresos"]))

(29.53847356777859, 30.774523779701518)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***ingreso*** dado la ***categoria_3***, es estadisticamente diferente de 0

In [ ]:
###Categoria 4
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==4]) - 1, loc=np.mean(data[data["estrategia"]==4]["ingresos"]), scale=stats.sem(data[data["estrategia"]==4]["ingresos"]))

(92.92968507218252, 95.8173028796247)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***ingreso*** dado la ***categoria_4***, es estadisticamente diferente de 0

### ***mercado_obj***

In [ ]:
#Ho: μ1 = μ2 = μ3 = μ4 = μ5
#Ha: al menos una μi es diferente

pg.anova(data=data, dv='mercado_obj', between='estrategia', detailed=True).round(3)

# p-value < 0,05

,Source,SS,DF,MS,F,p-unc,np2
0,estrategia,1.367870e+08,4,3.419675e+07,5964.272,0.0,0.927
1,Within,1.083650e+07,1890,5.733600e+03,NaN,NaN,NaN


Existe evidencia estadistica para inferir que al menos una media de la variable ***mercado_obj*** es estadisticamente diferentes entre si.

In [ ]:
###Categoria 0
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==0]) - 1, loc=np.mean(data[data["estrategia"]==0]["mercado_obj"]), scale=stats.sem(data[data["estrategia"]==0]["mercado_obj"]))

(48.78808578281867, 54.537881068010066)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***mercado_obj*** dado la ***categoria_0***, es estadisticamente diferente de 0

In [ ]:
###Categoria 1
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==1]) - 1, loc=np.mean(data[data["estrategia"]==1]["mercado_obj"]), scale=stats.sem(data[data["estrategia"]==1]["mercado_obj"]))

(90.69732068785079, 97.39334597881587)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***mercado_obj*** dado la ***categoria_1***, es estadisticamente diferente de 0

In [ ]:
###Categoria 2
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==2]) - 1, loc=np.mean(data[data["estrategia"]==2]["mercado_obj"]), scale=stats.sem(data[data["estrategia"]==2]["mercado_obj"]))

(650.3047220850498, 671.9302505925458)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***mercado_obj*** dado la ***categoria_2***, es estadisticamente diferente de 0

In [ ]:
###Categoria 3
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==3]) - 1, loc=np.mean(data[data["estrategia"]==3]["mercado_obj"]), scale=stats.sem(data[data["estrategia"]==3]["mercado_obj"]))

(269.18581164987444, 283.1537108965446)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***mercado_obj*** dado la ***categoria_3***, es estadisticamente diferente de 0

In [ ]:
###Categoria 4
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==4]) - 1, loc=np.mean(data[data["estrategia"]==4]["mercado_obj"]), scale=stats.sem(data[data["estrategia"]==4]["mercado_obj"]))

(660.6742172471294, 680.3956622709428)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***mercado_obj*** dado la ***categoria_4***, es estadisticamente diferente de 0

### ***rentabilidad_obj***

In [ ]:
#Ho: μ1 = μ2 = μ3 = μ4 = μ5
#Ha: al menos una μi es diferente

pg.anova(data=data, dv='rentabilidad_obj', between='estrategia', detailed=True).round(3)

# p-value < 0,05

,Source,SS,DF,MS,F,p-unc,np2
0,estrategia,84017.758,4,21004.440,4706.581,0.0,0.909
1,Within,8434.657,1890,4.463,NaN,NaN,NaN


Existe evidencia estadistica para inferir que al menos una media de la variable ***rentabilidad_obj*** es estadisticamente diferentes entre si.

In [ ]:
###Categoria 0
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==0]) - 1, loc=np.mean(data[data["estrategia"]==0]["rentabilidad_obj"]), scale=stats.sem(data[data["estrategia"]==0]["rentabilidad_obj"]))

/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


(nan, nan)

In [ ]:
np.mean(data[data["estrategia"]==0]["rentabilidad_obj"])

0.0

Existe evidencia estadistica para inferir que la media poblacional de la variable ***rentabilidad_obj*** dado la ***categoria_0***, no es estadisticamente diferente de 0

In [ ]:
###Categoria 1
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==1]) - 1, loc=np.mean(data[data["estrategia"]==1]["rentabilidad_obj"]), scale=stats.sem(data[data["estrategia"]==1]["rentabilidad_obj"]))

(2.3209950419860794, 2.6550049580139206)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***rentabilidad_obj*** dado la ***categoria_1***, es estadisticamente diferente de 0

In [ ]:
###Categoria 2
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==2]) - 1, loc=np.mean(data[data["estrategia"]==2]["rentabilidad_obj"]), scale=stats.sem(data[data["estrategia"]==2]["rentabilidad_obj"]))

(15.76658459072001, 16.304453660646107)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***rentabilidad_obj*** dado la ***categoria_2***, es estadisticamente diferente de 0

In [ ]:
###Categoria 3
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==3]) - 1, loc=np.mean(data[data["estrategia"]==3]["rentabilidad_obj"]), scale=stats.sem(data[data["estrategia"]==3]["rentabilidad_obj"]))

(6.267503109979297, 6.7404544497024)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***rentabilidad_obj*** dado la ***categoria_3***, es estadisticamente diferente de 0

In [ ]:
###Categoria 4
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==4]) - 1, loc=np.mean(data[data["estrategia"]==4]["rentabilidad_obj"]), scale=stats.sem(data[data["estrategia"]==4]["rentabilidad_obj"]))

(15.538526203416902, 16.0470159652578)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***rentabilidad_obj*** dado la ***categoria_4***, es estadisticamente diferente de 0

Dado que en el contraste de hipotesis de media poblacional en la variable rentabilidad_obj para el grupo 0 segun categoria, arroja un resultado (nan, nan), automaticamente la descartaremos

### ***empleados***

In [ ]:
#Ho: μ1 = μ2 = μ3 = μ4 = μ5
#Ha: al menos una μi es diferente

pg.anova(data=data, dv='empleados', between='estrategia', detailed=True).round(3)

# p-value < 0,05

,Source,SS,DF,MS,F,p-unc,np2
0,estrategia,8.956544e+09,4,2.239136e+09,10586.669,0.0,0.957
1,Within,3.997449e+08,1890,2.115053e+05,NaN,NaN,NaN


Existe evidencia estadistica para inferir que al menos una media de la variable ***empleados*** es estadisticamente diferentes entre si.

In [ ]:
###Categoria 0
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==0]) - 1, loc=np.mean(data[data["estrategia"]==0]["empleados"]), scale=stats.sem(data[data["estrategia"]==0]["empleados"]))

(170.2629682373615, 185.48841297810813)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***empleados*** dado la ***categoria_0***, es estadisticamente diferente de 0

In [ ]:
###Categoria 1
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==1]) - 1, loc=np.mean(data[data["estrategia"]==1]["empleados"]), scale=stats.sem(data[data["estrategia"]==1]["empleados"]))

(264.2851740016817, 291.39482599831825)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***empleados*** dado la ***categoria_1***, es estadisticamente diferente de 0

In [ ]:
###Categoria 2
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==2]) - 1, loc=np.mean(data[data["estrategia"]==2]["empleados"]), scale=stats.sem(data[data["estrategia"]==2]["empleados"]))

(4737.220815240811, 4880.314703884871)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***empleados*** dado la ***categoria_2***, es estadisticamente diferente de 0

In [ ]:
###Categoria 3
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==3]) - 1, loc=np.mean(data[data["estrategia"]==3]["empleados"]), scale=stats.sem(data[data["estrategia"]==3]["empleados"]))

(691.9728340087017, 704.244672622598)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***empleados*** dado la ***categoria_3***, es estadisticamente diferente de 0

In [ ]:
###Categoria 4
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==4]) - 1, loc=np.mean(data[data["estrategia"]==4]["empleados"]), scale=stats.sem(data[data["estrategia"]==4]["empleados"]))

(4704.798209421887, 4843.192152023897)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***empleados*** dado la ***categoria_4***, es estadisticamente diferente de 0

### ***rentabilidad_int***

In [ ]:
#Ho: μ1 = μ2 = μ3 = μ4 = μ5
#Ha: al menos una μi es diferente

pg.anova(data=data, dv='rentabilidad_int', between='estrategia', detailed=True).round(3)

# p-value < 0,05

,Source,SS,DF,MS,F,p-unc,np2
0,estrategia,55380.145,4,13845.036,5382.176,0.0,0.919
1,Within,4861.810,1890,2.572,NaN,NaN,NaN


Existe evidencia estadistica para inferir que al menos una media de la variable ***rentabilidad_int*** es estadisticamente diferentes entre si.

In [ ]:
###Categoria 0
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==0]) - 1, loc=np.mean(data[data["estrategia"]==0]["rentabilidad_int"]), scale=stats.sem(data[data["estrategia"]==0]["rentabilidad_int"]))

(2.962909096259176, 3.1254886937960724)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***rentabilidad_int*** dado la ***categoria_0***, es estadisticamente diferente de 0

In [ ]:
###Categoria 1
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==1]) - 1, loc=np.mean(data[data["estrategia"]==1]["rentabilidad_int"]), scale=stats.sem(data[data["estrategia"]==1]["rentabilidad_int"]))

(3.397437501290296, 3.498562498709704)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***rentabilidad_int*** dado la ***categoria_1***, es estadisticamente diferente de 0

In [ ]:
###Categoria 2
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==2]) - 1, loc=np.mean(data[data["estrategia"]==2]["rentabilidad_int"]), scale=stats.sem(data[data["estrategia"]==2]["rentabilidad_int"]))

(14.775374569111003, 15.18637406476878)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***rentabilidad_int*** dado la ***categoria_2***, es estadisticamente diferente de 0

In [ ]:
###Categoria 3
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==3]) - 1, loc=np.mean(data[data["estrategia"]==3]["rentabilidad_int"]), scale=stats.sem(data[data["estrategia"]==3]["rentabilidad_int"]))

(5.777753427431797, 6.174501214478017)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***rentabilidad_int*** dado la ***categoria_3***, es estadisticamente diferente de 0

In [ ]:
###Categoria 4
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==4]) - 1, loc=np.mean(data[data["estrategia"]==4]["rentabilidad_int"]), scale=stats.sem(data[data["estrategia"]==4]["rentabilidad_int"]))

(14.761923875803808, 15.146509859135952)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***rentabilidad_int*** dado la ***categoria_4***, es estadisticamente diferente de 0

### ***participacion***

In [ ]:
#Ho: μ1 = μ2 = μ3 = μ4 = μ5
#Ha: al menos una μi es diferente

pg.anova(data=data, dv='participacion', between='estrategia', detailed=True).round(3)

# p-value < 0,05

,Source,SS,DF,MS,F,p-unc,np2
0,estrategia,160683.849,4,40170.962,2087.412,0.0,0.815
1,Within,36371.885,1890,19.244,NaN,NaN,NaN


Existe evidencia estadistica para inferir que al menos una media de la variable ***participacion*** es estadisticamente diferentes entre si.

In [ ]:
###Categoria 0
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==0]) - 1, loc=np.mean(data[data["estrategia"]==0]["participacion"]), scale=stats.sem(data[data["estrategia"]==0]["participacion"]))

(12.730230764991076, 14.054299621749257)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***participacion*** dado la ***categoria_0***, es estadisticamente diferente de 0

In [ ]:
###Categoria 1
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==1]) - 1, loc=np.mean(data[data["estrategia"]==1]["participacion"]), scale=stats.sem(data[data["estrategia"]==1]["participacion"]))

(3.858119886337108, 4.136546780329558)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***participacion*** dado la ***categoria_1***, es estadisticamente diferente de 0

In [ ]:
###Categoria 2
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==2]) - 1, loc=np.mean(data[data["estrategia"]==2]["participacion"]), scale=stats.sem(data[data["estrategia"]==2]["participacion"]))

(25.34877607975114, 26.410786761778915)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***participacion*** dado la ***categoria_2***, es estadisticamente diferente de 0

In [ ]:
###Categoria 3
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==3]) - 1, loc=np.mean(data[data["estrategia"]==3]["participacion"]), scale=stats.sem(data[data["estrategia"]==3]["participacion"]))

(7.833599649670341, 8.118654992239472)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***participacion*** dado la ***categoria_3***, es estadisticamente diferente de 0

In [ ]:
###Categoria 4
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==4]) - 1, loc=np.mean(data[data["estrategia"]==4]["participacion"]), scale=stats.sem(data[data["estrategia"]==4]["participacion"]))

(25.856756379437403, 26.822761692851753)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***participacion*** dado la ***categoria_4***, es estadisticamente diferente de 0

### ***eficiencia***

In [ ]:
#Ho: μ1 = μ2 = μ3 = μ4 = μ5
#Ha: al menos una μi es diferente

pg.anova(data=data, dv='eficiencia', between='estrategia', detailed=True).round(3)

# p-value < 0,05

,Source,SS,DF,MS,F,p-unc,np2
0,estrategia,56449.017,4,14112.254,5510.389,0.0,0.921
1,Within,4840.341,1890,2.561,NaN,NaN,NaN


Existe evidencia estadistica para inferir que al menos una media de la variable ***eficiencia*** es estadisticamente diferentes entre si.

In [ ]:
###Categoria 0
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==0]) - 1, loc=np.mean(data[data["estrategia"]==0]["eficiencia"]), scale=stats.sem(data[data["estrategia"]==0]["eficiencia"]))

(2.8553269157146275, 3.028650984837859)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***eficiencia*** dado la ***categoria_0***, es estadisticamente diferente de 0

In [ ]:
###Categoria 1
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==1]) - 1, loc=np.mean(data[data["estrategia"]==1]["eficiencia"]), scale=stats.sem(data[data["estrategia"]==1]["eficiencia"]))

(3.4585040121972743, 3.560162654469392)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***eficiencia*** dado la ***categoria_1***, es estadisticamente diferente de 0

In [ ]:
###Categoria 2
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==2]) - 1, loc=np.mean(data[data["estrategia"]==2]["eficiencia"]), scale=stats.sem(data[data["estrategia"]==2]["eficiencia"]))

(14.877974952158707, 15.285959474070802)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***eficiencia*** dado la ***categoria_2***, es estadisticamente diferente de 0

In [ ]:
###Categoria 3
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==3]) - 1, loc=np.mean(data[data["estrategia"]==3]["eficiencia"]), scale=stats.sem(data[data["estrategia"]==3]["eficiencia"]))

(5.7966353851926735, 6.198059575019529)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***eficiencia*** dado la ***categoria_3***, es estadisticamente diferente de 0

In [ ]:
###Categoria 4
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==4]) - 1, loc=np.mean(data[data["estrategia"]==4]["eficiencia"]), scale=stats.sem(data[data["estrategia"]==4]["eficiencia"]))

(14.852559629911747, 15.229368080931627)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***eficiencia*** dado la ***categoria_4***, es estadisticamente diferente de 0

### ***deuda***

In [ ]:
#Ho: μ1=μ2=μ3=μ4=μ5
#Ha: al menos una μi es diferente

pg.anova(data=data, dv='deuda', between='estrategia', detailed=True).round(3)

# p-value < 0,05

,Source,SS,DF,MS,F,p-unc,np2
0,estrategia,2885873.582,4,721468.396,6830.797,0.0,0.935
1,Within,199621.692,1890,105.620,NaN,NaN,NaN


Existe evidencia estadistica para inferir que al menos una media
de la variable ***deuda*** es estadisticamente diferentes entre si.

In [ ]:
###Categoria 0
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==0]) - 1, loc=np.mean(data[data["estrategia"]==0]["deuda"]), scale=stats.sem(data[data["estrategia"]==0]["deuda"]))

(4.86885304215316, 5.451588946797117)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***deuda*** dado la ***categoria_0***, es estadisticamente diferente de 0

In [ ]:
###Categoria 1
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==1]) - 1, loc=np.mean(data[data["estrategia"]==1]["deuda"]), scale=stats.sem(data[data["estrategia"]==1]["deuda"]))

(14.567033346033732, 15.736966653966267)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***deuda*** dado la ***categoria_1***, es estadisticamente diferente de 0

In [ ]:
###Categoria 2
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==2]) - 1, loc=np.mean(data[data["estrategia"]==2]["deuda"]), scale=stats.sem(data[data["estrategia"]==2]["deuda"]))

(92.00198507287224, 94.98708596537911)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***deuda*** dado la ***categoria_2***, es estadisticamente diferente de 0

In [ ]:
###Categoria 3
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==3]) - 1, loc=np.mean(data[data["estrategia"]==3]["deuda"]), scale=stats.sem(data[data["estrategia"]==3]["deuda"]))

(29.361711840966915, 30.622373039669693)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***deuda*** dado la ***categoria_3***, es estadisticamente diferente de 0

In [ ]:
###Categoria 4
#Ho: μ = 0
#Ha: μ ≠ 0

stats.t.interval(0.95, len(data[data["estrategia"]==4]) - 1, loc=np.mean(data[data["estrategia"]==4]["deuda"]), scale=stats.sem(data[data["estrategia"]==4]["deuda"]))

(94.11664750350813, 97.00624406275692)

Existe evidencia estadistica para inferir que la media poblacional de la variable ***deuda*** dado la ***categoria_4***, es estadisticamente diferente de 0

# ***Creacion segunda base de datos***

## ***Matriz de correlacion de Pearson***

In [ ]:
data[["estrategia" ,"ingresos", "mercado_obj", "empleados", "rentabilidad_int", "participacion", "eficiencia", "deuda"]].corr()

,estrategia,ingresos,mercado_obj,empleados,rentabilidad_int,participacion,eficiencia,deuda
estrategia,1.000000,0.689345,0.728297,0.625300,0.671851,0.434656,0.674360,0.696219
ingresos,0.689345,1.000000,0.927994,0.940651,0.928055,0.781054,0.926939,0.934082
mercado_obj,0.728297,0.927994,1.000000,0.921108,0.917377,0.775403,0.921226,0.928005
empleados,0.625300,0.940651,0.921108,1.000000,0.932953,0.832672,0.935162,0.936920
rentabilidad_int,0.671851,0.928055,0.917377,0.932953,1.000000,0.799149,0.917927,0.927822
participacion,0.434656,0.781054,0.775403,0.832672,0.799149,1.000000,0.799770,0.790326
eficiencia,0.674360,0.926939,0.921226,0.935162,0.917927,0.799770,1.000000,0.927614
deuda,0.696219,0.934082,0.928005,0.936920,0.927822,0.790326,0.927614,1.000000


## ***Matriz de correlacion de Spearman***

In [ ]:
data[["estrategia" ,"ingresos", "mercado_obj", "empleados", "rentabilidad_int", "participacion", "eficiencia", "deuda"]].corr(method='spearman')

,estrategia,ingresos,mercado_obj,empleados,rentabilidad_int,participacion,eficiencia,deuda
estrategia,1.000000,0.765120,0.771823,0.754089,0.705265,0.455346,0.705598,0.778218
ingresos,0.765120,1.000000,0.885624,0.878296,0.848173,0.647197,0.850931,0.885182
mercado_obj,0.771823,0.885624,1.000000,0.870818,0.848352,0.673374,0.848613,0.885260
empleados,0.754089,0.878296,0.870818,1.000000,0.846419,0.690029,0.845222,0.874050
rentabilidad_int,0.705265,0.848173,0.848352,0.846419,1.000000,0.701744,0.824418,0.851953
participacion,0.455346,0.647197,0.673374,0.690029,0.701744,1.000000,0.694461,0.654346
eficiencia,0.705598,0.850931,0.848613,0.845222,0.824418,0.694461,1.000000,0.851059
deuda,0.778218,0.885182,0.885260,0.874050,0.851953,0.654346,0.851059,1.000000


## ***Matriz de correlacion de Kendall***

In [ ]:
data[["estrategia" ,"ingresos", "mercado_obj", "empleados", "rentabilidad_int", "participacion", "eficiencia", "deuda"]].corr(method='kendall')

,estrategia,ingresos,mercado_obj,empleados,rentabilidad_int,participacion,eficiencia,deuda
estrategia,1.000000,0.608061,0.603052,0.577522,0.558007,0.326010,0.562010,0.622613
ingresos,0.608061,1.000000,0.684027,0.671634,0.659864,0.443508,0.662374,0.687687
mercado_obj,0.603052,0.684027,1.000000,0.652983,0.653860,0.473480,0.656570,0.683611
empleados,0.577522,0.671634,0.652983,1.000000,0.651075,0.487754,0.650638,0.664822
rentabilidad_int,0.558007,0.659864,0.653860,0.651075,1.000000,0.514361,0.649507,0.665566
participacion,0.326010,0.443508,0.473480,0.487754,0.514361,1.000000,0.508337,0.455772
eficiencia,0.562010,0.662374,0.656570,0.650638,0.649507,0.508337,1.000000,0.663200
deuda,0.622613,0.687687,0.683611,0.664822,0.665566,0.455772,0.663200,1.000000


Esto lo realizamos para dividir las variables explicativas politomicas a variables explicativas ***dicotomicas***. Para estos efectos, 1 significa que se cumple la condicion, y 0 es cualquier caso contrario.

### ***Informacion variables explicativas***

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1895 entries, 0 to 1894
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   estrategia        1895 non-null   int64
 1   ingresos          1895 non-null   int64
 2   mercado_obj       1895 non-null   int64
 3   rentabilidad_obj  1895 non-null   int64
 4   conectividad      1895 non-null   int64
 5   politico          1895 non-null   int64
 6   empleados         1895 non-null   int64
 7   rentabilidad_int  1895 non-null   int64
 8   participacion     1895 non-null   int64
 9   eficiencia        1895 non-null   int64
 10  deuda             1895 non-null   int64
dtypes: int64(11)
memory usage: 163.0 KB


### ***Elaboracion 2da base de datos***

In [ ]:
data2 = pd.DataFrame({
    "estrategia":data["estrategia"],
    "ingresos":data["ingresos"],
    "mercado_obj":data["mercado_obj"],
    "conectividad_0":np.where(data["conectividad"]==0,1,0),
    "conectividad_1":np.where(data["conectividad"]==1,1,0),
    "conectividad_2":np.where(data["conectividad"]==2,1,0),
    "conectividad_3":np.where(data["conectividad"]==3,1,0),
    "politico_0":np.where(data["politico"]==0,1,0),
    "politico_1":np.where(data["politico"]==1,1,0),
    "politico_2":np.where(data["politico"]==2,1,0),
    "politico_3":np.where(data["politico"]==3,1,0),
    "empleados":data["empleados"],
    "rentabilidad_int":data["rentabilidad_int"],
    "participacion":data["participacion"],
    "eficiencia":data["eficiencia"],
    "deuda":data["deuda"]
});data2

,estrategia,ingresos,mercado_obj,conectividad_0,conectividad_1,conectividad_2,conectividad_3,politico_0,politico_1,politico_2,politico_3,empleados,rentabilidad_int,participacion,eficiencia,deuda
0,3,38,354,0,0,1,0,0,0,1,0,649,9,8,8,28
1,2,80,781,0,0,0,1,1,0,0,0,3866,14,21,12,91
2,4,91,563,0,0,0,1,0,1,0,0,5254,14,34,17,119
3,0,8,10,0,1,0,0,0,0,0,1,166,3,10,3,8
4,0,8,58,0,0,1,0,0,1,0,0,270,4,18,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,4,95,589,0,0,0,1,0,1,0,0,4386,13,26,14,80
1891,1,20,123,0,1,0,0,0,0,0,1,132,3,6,3,11
1892,3,24,360,0,0,1,0,0,1,0,0,652,9,10,5,40
1893,4,105,531,0,0,0,0,1,0,0,0,5643,13,18,15,100


# ***Generacion de combinaciones con variables explicativas***

In [ ]:
variables = list(data2[["ingresos", "mercado_obj", "conectividad_0", "conectividad_1", "conectividad_2", "conectividad_3", "politico_0", "politico_1", "politico_2", "politico_3", "empleados", "rentabilidad_int", "participacion", "eficiencia", "deuda"]].columns)

combinaciones = []
for i in range(1, len(variables)+1):
  combinaciones.extend(list(itertools.combinations(variables,i)))
combinaciones

[('ingresos',),
 ('mercado_obj',),
 ('conectividad_0',),
 ('conectividad_1',),
 ('conectividad_2',),
 ('conectividad_3',),
 ('politico_0',),
 ('politico_1',),
 ('politico_2',),
 ('politico_3',),
 ('empleados',),
 ('rentabilidad_int',),
 ('participacion',),
 ('eficiencia',),
 ('deuda',),
 ('ingresos', 'mercado_obj'),
 ('ingresos', 'conectividad_0'),
 ('ingresos', 'conectividad_1'),
 ('ingresos', 'conectividad_2'),
 ('ingresos', 'conectividad_3'),
 ('ingresos', 'politico_0'),
 ('ingresos', 'politico_1'),
 ('ingresos', 'politico_2'),
 ('ingresos', 'politico_3'),
 ('ingresos', 'empleados'),
 ('ingresos', 'rentabilidad_int'),
 ('ingresos', 'participacion'),
 ('ingresos', 'eficiencia'),
 ('ingresos', 'deuda'),
 ('mercado_obj', 'conectividad_0'),
 ('mercado_obj', 'conectividad_1'),
 ('mercado_obj', 'conectividad_2'),
 ('mercado_obj', 'conectividad_3'),
 ('mercado_obj', 'politico_0'),
 ('mercado_obj', 'politico_1'),
 ('mercado_obj', 'politico_2'),
 ('mercado_obj', 'politico_3'),
 ('mercado_obj

# ***Analisis multicolinealidad***

 ### ***Matriz de correlación entre las variables explicativas del conjunto de datos***

In [ ]:
v_explicativas = data2[["ingresos", "mercado_obj", "conectividad_0", "conectividad_1", "conectividad_2", "conectividad_3", "politico_0", "politico_1", "politico_2", "politico_3", "empleados", "rentabilidad_int", "participacion", "eficiencia", "deuda"]]
v_explicativas = (v_explicativas) ##sm.add_constant

In [ ]:
print(v_explicativas.corr())

                  ingresos  mercado_obj  conectividad_0  conectividad_1  \
ingresos          1.000000     0.927994       -0.274037       -0.438515   
mercado_obj       0.927994     1.000000       -0.277256       -0.440497   
conectividad_0   -0.274037    -0.277256        1.000000       -0.137068   
conectividad_1   -0.438515    -0.440497       -0.137068        1.000000   
conectividad_2   -0.440738    -0.434689       -0.139746       -0.299250   
conectividad_3    0.433639     0.440289       -0.151261       -0.323909   
politico_0        0.298821     0.276523       -0.063065       -0.159058   
politico_1        0.273737     0.284375       -0.049754       -0.133746   
politico_2       -0.283491    -0.262176        0.029223        0.101714   
politico_3       -0.299057    -0.288568        0.041928        0.189044   
empleados         0.940651     0.921108       -0.230376       -0.449192   
rentabilidad_int  0.928055     0.917377       -0.247507       -0.444928   
participacion     0.78105

### ***Primeras Variables explicativas escogidas***

La combinacion con la correlacion mas baja es la de la variable ***conectividad_0*** y ***politico_2***. Estas variables las usaremos para la combinacion de modelos.

## ***Resultado VIF de cada variable***

encontrar la matriz de diseño para el modelo de regresión lineal usando ***estrategia*** como variable de respuesta

In [ ]:
y, X = dmatrices('estrategia ~ ingresos + mercado_obj + C(conectividad_0) + C(conectividad_1) + C(conectividad_2) + C(conectividad_3) + C(politico_0) + C(politico_1) + C(politico_2) + C(politico_3) + empleados + rentabilidad_int + participacion + eficiencia + deuda', data=data2, return_type='dataframe')

## Calculo VIF para cada variable explicativa
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['variable'] = X.columns

## Resultado vif por variable, jerarquizado de menor a mayor
print(vif.sort_values(by='VIF'))


          VIF                variable
4    1.872258  C(conectividad_3)[T.1]
7    2.326620      C(politico_2)[T.1]
8    2.448970      C(politico_3)[T.1]
1    2.871572  C(conectividad_0)[T.1]
13   3.588285           participacion
6    4.215533      C(politico_1)[T.1]
5    4.323127      C(politico_0)[T.1]
2    5.736730  C(conectividad_1)[T.1]
3    5.759841  C(conectividad_2)[T.1]
10  10.800575             mercado_obj
12  10.961501        rentabilidad_int
14  11.271100              eficiencia
15  12.797937                   deuda
9   13.269598                ingresos
11  15.774842               empleados
0   56.388716               Intercept


Utilizaremos las variables con ***VIF*** menor a 10, ya que si es mayor a 10, Xj es altamento colineal. Por tanto, junto a la combinacion de correlacion mas baja, incluiremos la variable cuantitativa ***participacion***, ya que es la que tiene el VIF mas bajo, entre las variables cuantitativas

# ***Elaboracion combinaciones de modelos de Regresion logistica multinomial***

## ***Correlacion variables explicativas***

In [ ]:
data2[["conectividad_0","politico_2","participacion"]].corr()

,conectividad_0,politico_2,participacion
conectividad_0,1.000000,0.029223,-0.075983
politico_2,0.029223,1.000000,-0.276974
participacion,-0.075983,-0.276974,1.000000


In [ ]:
data2[["conectividad_0","politico_2","participacion"]].corr(method='spearman')

,conectividad_0,politico_2,participacion
conectividad_0,1.000000,0.029223,-0.055502
politico_2,0.029223,1.000000,-0.257563
participacion,-0.055502,-0.257563,1.000000


In [ ]:
data2[["conectividad_0","politico_2","participacion"]].corr(method='kendall')

,conectividad_0,politico_2,participacion
conectividad_0,1.000000,0.029223,-0.046205
politico_2,0.029223,1.000000,-0.214418
participacion,-0.046205,-0.214418,1.000000


## ***Elaboracion modelos de probabilidad logistica multinomial con variables explicativas seleccionadas***

In [ ]:
variables = list(data2[["conectividad_0","politico_2","participacion"]].columns)

combinaciones = []
for i in range(1, len(variables)+1):
  combinaciones.extend(list(itertools.combinations(variables,i)))
#combinaciones

Para efecto de las combinaciones solo usamos las variables ***conectividad_0, politico_2*** y ***participacion***.

## ***Ajuste diferentes modelos, y calculo de AIC y BIC, respectivamente***

In [ ]:
resultadosMNL = []
resultadosMNL_AIC = []
resultadosMNL_BIC = []

for j in combinaciones:
  X = data2[list(j)]
  X = sm.add_constant(X)
  y = data2[["estrategia"]]

  modelo = sm.MNLogit(y, X).fit()
  resultadosMNL.append((j, modelo))
  aic = modelo.aic
  resultadosMNL_AIC.append(aic)
  bic = modelo.bic
  resultadosMNL_BIC.append(bic)

         Current function value: 1.499830
         Iterations: 35
         Current function value: 1.536758
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.812175
         Iterations 10


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: 1.428739
         Iterations: 35
         Current function value: 0.709943
         Iterations: 35


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: 0.792246
         Iterations: 35
         Current function value: 0.692791
         Iterations: 35


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Jerarquizado a traves del criterio ***BIC***

In [ ]:
seleccion = pd.DataFrame({"Modelo":combinaciones, "MNL_AIC":resultadosMNL_AIC,
                          "MNL_BIC":resultadosMNL_BIC})
seleccion = seleccion.sort_values(by = "MNL_BIC", ascending = True) ##sort_values ORDENAR DE MEJOR A PEOR¡
seleccion

,Modelo,MNL_AIC,MNL_BIC
6,"(conectividad_0, politico_2, participacion)",2657.676026,2746.427612
4,"(conectividad_0, participacion)",2714.683044,2781.246733
5,"(politico_2, participacion)",3026.612969,3093.176659
2,"(participacion,)",3094.142107,3138.517900
3,"(conectividad_0, politico_2)",5438.919826,5505.483515
0,"(conectividad_0,)",5700.354948,5744.730741
1,"(politico_2,)",5840.314483,5884.690276


### ***Modelo seleccionado por menor BIC***

In [ ]:
modelo_seleccionado = resultadosMNL[seleccion.index[0]][1]
print(modelo_seleccionado.summary())

                          MNLogit Regression Results                          
Dep. Variable:             estrategia   No. Observations:                 1895
Model:                        MNLogit   Df Residuals:                     1879
Method:                           MLE   Df Model:                           12
Date:                Tue, 05 Dec 2023   Pseudo R-squ.:                  0.5692
Time:                        17:55:05   Log-Likelihood:                -1312.8
converged:                      False   LL-Null:                       -3047.6
Covariance Type:            nonrobust   LLR p-value:                     0.000
  estrategia=1       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             12.0043      0.694     17.304      0.000      10.645      13.364
conectividad_0   -31.0269   6.91e+04     -0.000      1.000   -1.36e+05    1.35e+05
politico_2         0.4229      0.362

### ***Efectos marginales para el modelo MNL seleccionado***

In [ ]:
efectos_marginalesMNL1 = modelo_seleccionado.get_margeff()
print(efectos_marginalesMNL1.summary())

       MNLogit Marginal Effects      
Dep. Variable:             estrategia
Method:                          dydx
At:                           overall
  estrategia=0      dy/dx    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
conectividad_0     2.2721   7875.169      0.000      1.000   -1.54e+04    1.54e+04
politico_2         1.1165   1.41e+08   7.94e-09      1.000   -2.75e+08    2.75e+08
participacion      0.0101      0.001      8.005      0.000       0.008       0.013
----------------------------------------------------------------------------------
  estrategia=1      dy/dx    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
conectividad_0    -0.1956   6629.167  -2.95e-05      1.000    -1.3e+04     1.3e+04
politico_2         0.0045   1.72e+05   2.63e-08      1.000   -3.37e+05    3.37e+05
participacion     

### ***Odds_ratio***

In [ ]:
# Calcular los Odds Ratio
odds_ratio = np.exp(modelo_seleccionado.params)
# Mostrar los Odds Ratio
print("Odds Ratio:")
print(odds_ratio)
# Calcular los errores estándar de los coeficientes
errores_estandar = modelo_seleccionado.bse
# Calcular los Odds Ratio estandarizados
odds_ratio_estandarizados = np.exp(modelo_seleccionado.params) ** errores_estandar

# Mostrar los Odds Ratio estandarizados
print("Odds Ratio Estandarizados:")
print(odds_ratio_estandarizados)

Odds Ratio:
                           0             1             2             3
const           1.634530e+05  3.276539e-04  4.788891e+01  2.362793e-04
conectividad_0  3.351080e-14  5.514833e-11  8.928552e-14  1.004050e-09
politico_2      1.526340e+00  1.805450e-22  1.483632e+00  7.671213e-08
participacion   1.698414e-01  1.535102e+00  7.028576e-01  1.561942e+00
Odds Ratio Estandarizados:
                          0         1         2         3
const           4136.797369  0.003526  3.520338  0.002679
conectividad_0     0.000000  0.000000  0.000000  0.000000
politico_2         1.165241  0.000000  1.103242  0.000000
participacion      0.819790  1.015284  0.988537  1.015987


## ***Prediccion modelo***

Para efectos de prediccion, se elaborara en base al registro de la fila numero 633 de la base de datos. El cual fue escogido al azar.

In [ ]:
data2.loc[632, :]

estrategia            3
ingresos             38
mercado_obj         278
conectividad_0        0
conectividad_1        0
conectividad_2        1
conectividad_3        0
politico_0            0
politico_1            0
politico_2            1
politico_3            0
empleados           604
rentabilidad_int      3
participacion         7
eficiencia            3
deuda                35
Name: 632, dtype: int64

Tomaremos los valores que nos serviran en nuestro modelo

#### ***Indicamos valores***

In [ ]:
x1_test = 0 ## conectividad_0
x2_test = 1 ## politico_2
x3_test = 7 ## participacion

X_new = pd.DataFrame({'const': [1], 'conectividad_0': [x1_test], 'politico_2': [x2_test], 'participacion': [x3_test]})

#### ***Probabilidad de pertenencia a cada categoria***

In [ ]:
predicted_probabilities = modelo_seleccionado.predict(X_new)
print(f"Probabilidades de pertenencia a cada una de las categorías:\n{predicted_probabilities.loc[0].round(4)}")

Probabilidades de pertenencia a cada una de las categorías:
0    0.1244
1    0.1265
2    0.0000
3    0.7490
4    0.0000
Name: 0, dtype: float64


#### ***Categoria Predicha (considerando la mas probable)***

In [ ]:
predicted_category = np.argmax(predicted_probabilities.values)

#### ***Indicamos categoria predicha***

In [ ]:
print(f"Categoría predicha para un empresa en la cual no tiene conectividad en zona objetivo de expansion (conectividad_0 = {x1_test}), tiene confictos")
print(f"politicos moderados en zona objetivo de expansion (politico_2 = {x2_test}) y tiene un porcentaje de participacion de mercado de {x3_test}% es: Categoria", predicted_category)

print()
if predicted_category == 0:
    print("Por tanto, la empresa tomó una estrategia de consolidación y optimización (la empresa decidio no expandirse)")
elif predicted_category == 1:
    print("Por tanto, la empresa asumió una estrategia de enfoque de mercado local (la empresa decidio abrir nuevas sucursales dentro de su ciudad)")
elif predicted_category == 2:
    print("Por tanto, la empresa tomó una estrategia de expansión regional (la empresa decidio abrir nuevas sucursales en otras regiones)")
elif predicted_category == 3:
    print("Por tanto, la empresa tomó una estrategia de expansión continental (la empresa decidio abrir nuevas sucursales en paises del continente al que pertenece)")
else:
    print("Por tanto, la empresa asumió una estrategia de expansión internacional (la empresa decidio abrir nuevas sucursales en pa´ıses de otros continentes)")


Categoría predicha para un empresa en la cual no tiene conectividad en zona objetivo de expansion (conectividad_0 = 0), tiene confictos
politicos moderados en zona objetivo de expansion (politico_2 = 1) y tiene un porcentaje de participacion de mercado de 7% es: Categoria 3

Por tanto, la empresa tomó una estrategia de expansión continental (la empresa decidio abrir nuevas sucursales en paises del continente al que pertenece)
